In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!cp "drive/My Drive/Colab Notebooks/cracks/models.py" .
!cp "drive/My Drive/Colab Notebooks/cracks/losses.py" .
!cp "drive/My Drive/Colab Notebooks/cracks/utils.py" .
!cp "drive/My Drive/Colab Notebooks/cracks/dataset.csv" .

In [3]:
!pip install -U segmentation-models

     |████████████████████████████████| 51kB 2.4MB/s 


In [4]:
import pandas as pd

import models
import losses
import utils
import json
from tensorflow.keras.optimizers import RMSprop, Adam
from matplotlib import pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

In [5]:
%env SM_FRAMEWORK=tf.keras
import segmentation_models as sm

env: SM_FRAMEWORK=tf.keras
Segmentation Models: using `tf.keras` framework.


In [6]:
INPUT_SHAPE2 = (544, 1984, 1)
#INPUT_SHAPE2 = (1056, 3936, 1) #16
#INPUT_SHAPE2 = (1056, 3888, 1) #8
#INPUT_SHAPE1 = (352, 1152, 1)
VALIDATION_SPLIT = 0.2
CHECKPOINT_PATH = 'drive/My Drive/Colab Notebooks/cracks/model.h5'
TEST_PATH = 'drive/My Drive/Colab Notebooks/cracks/test'
BATCH_SIZE = 5

## Инициализация модели и загрузка весов

In [7]:
#model = sm.Unet('resnet18',input_shape=INPUT_SHAPE2,activation='softmax',encoder_weights=None, classes=3)
model = models.Unet(INPUT_SHAPE2, 3)
model.compile(RMSprop(), loss=sm.losses.DiceLoss(), metrics = [sm.metrics.IOUScore()])
model.load_weights(CHECKPOINT_PATH)

# Загрузка данных, которые не участвовали в обучении, получение результатов модели

In [8]:
df = pd.read_csv('dataset.csv')

test_idxs = [21, 65, 75, 100]
test_blur_contr = df.iloc[test_idxs]
df.drop(test_idxs, inplace = True)

train, val = train_test_split(df, test_size=0.15, random_state=666)

test_df = pd.concat([test_blur_contr, val])
X = test_df['Labeled Data']
Y = test_df['Label']

val_gen = utils.genData(X, Y, 1, INPUT_SHAPE2, augmentations=None) # генератор для валидации

# Визуализация и сохранение результатов

In [9]:
import cv2 as cv

def draw_mask(mask, orig, sufix):
  utils.drawWidth(mask[:,:,1], orig, type = 'crack')
  utils.drawWidth(mask[:,:,2], orig, type = 'gap')
  cv.imwrite(TEST_PATH + sufix, orig)

for i in range(len(X)):
  (x, y) = next(val_gen)
  predict = model.predict(x)[0]
  predict = utils.softmax_to_onehot(predict).astype('uint8') * 255

  orig = utils.get_image_by_url(X.to_list()[i], None, 'original')
  orig = cv.cvtColor(orig, cv.COLOR_GRAY2RGB).astype('uint8')
  
  draw_mask(predict, orig.copy(), '/{}_predict.png'.format(i))
  draw_mask(y[0].astype('uint8') * 255, orig, '/{}_truth.png'.format(i))

KeyboardInterrupt: ignored